# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [22]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [37]:
companyName = "V&J"
companyURL = "https://vjfurnishings.com/"

In [38]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website(companyURL)
huggingface.links

['https://hacklink.market/',
 'https://spyhackerz.org/forum/',
 'https://trafik.market/',
 'https://kaysericastajans.club/',
 'https://vjfurnishings.com/',
 'https://vjfurnishings.com/about-us-vj-furnishings/',
 'https://vjfurnishings.com/infrastructure/',
 'https://vjfurnishings.com/brands/',
 'https://vjfurnishings.com/product-category/brands-amber/',
 'https://vjfurnishings.com/product/',
 'https://vjfurnishings.com/product-category/curtains-sheers-collection/',
 'https://vjfurnishings.com/product-category/upholstery-fabric/',
 'https://vjfurnishings.com/product-category/leatherette/',
 'https://vjfurnishings.com/product-category/indoor-outdoor/',
 'https://vjfurnishings.com/news-event/',
 'https://vjfurnishings.com/careers/',
 'https://vjfurnishings.com/store-locator/',
 'https://vjfurnishings.com/our-network/',
 'https://vjfurnishings.com/amber/',
 'https://vjfurnishings.com/corporate/',
 'https://vjfurnishings.com/contact-us/',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#',
 '#'

In [39]:
get_links(companyURL)

{'links': [{'type': 'about page',
   'url': 'https://vjfurnishings.com/about-us-vj-furnishings/'},
  {'type': 'careers page', 'url': 'https://vjfurnishings.com/careers/'},
  {'type': 'company page', 'url': 'https://vjfurnishings.com/corporate/'},
  {'type': 'infrastructure page',
   'url': 'https://vjfurnishings.com/infrastructure/'},
  {'type': 'brands page', 'url': 'https://vjfurnishings.com/brands/'},
  {'type': 'news and events page',
   'url': 'https://vjfurnishings.com/news-event/'},
  {'type': 'our network page',
   'url': 'https://vjfurnishings.com/our-network/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [40]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [41]:
print(get_all_details(companyURL))

Found links: {'links': [{'type': 'about page', 'url': 'https://vjfurnishings.com/about-us-vj-furnishings/'}, {'type': 'careers page', 'url': 'https://vjfurnishings.com/careers/'}, {'type': 'corporate page', 'url': 'https://vjfurnishings.com/corporate/'}, {'type': 'infrastructure page', 'url': 'https://vjfurnishings.com/infrastructure/'}, {'type': 'our network page', 'url': 'https://vjfurnishings.com/our-network/'}, {'type': 'contact page', 'url': 'https://vjfurnishings.com/contact-us/'}]}
Landing page:
Webpage Title:
V&J Furnishings - Upholstery | Curtain & Sheers | Artificial Leather
Webpage Contents:
HOME
ABOUT US
INFRASTRUCTURE
BRANDS
AMBER
PRODUCT
CURTAINS & SHEERS
UPHOLSTERY FABRIC
LEATHERETTE
OUTDOOR / INDOOR
NEWS & EVENT
CAREERS
STORE LOCATOR
V&J FURNISHINGS
AMBER
CORPORATE
CONTACT US
Millions of people around the world visit Envato to buy and sell creative assets, use smart design templates, learn creative skills or even hire freelancers. With an industry-leading marketplace pa

In [42]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [43]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [44]:
get_brochure_user_prompt(companyName, companyURL)

Found links: {'links': [{'type': 'about page', 'url': 'https://vjfurnishings.com/about-us-vj-furnishings/'}, {'type': 'infrastructure page', 'url': 'https://vjfurnishings.com/infrastructure/'}, {'type': 'careers page', 'url': 'https://vjfurnishings.com/careers/'}, {'type': 'corporate page', 'url': 'https://vjfurnishings.com/corporate/'}, {'type': 'our network page', 'url': 'https://vjfurnishings.com/our-network/'}, {'type': 'brands page', 'url': 'https://vjfurnishings.com/brands/'}]}


'You are looking at a company called: V&J\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nV&J Furnishings - Upholstery | Curtain & Sheers | Artificial Leather\nWebpage Contents:\nHOME\nABOUT US\nINFRASTRUCTURE\nBRANDS\nAMBER\nPRODUCT\nCURTAINS & SHEERS\nUPHOLSTERY FABRIC\nLEATHERETTE\nOUTDOOR / INDOOR\nNEWS & EVENT\nCAREERS\nSTORE LOCATOR\nV&J FURNISHINGS\nAMBER\nCORPORATE\nCONTACT US\nMillions of people around the world visit Envato to buy and sell creative assets, use smart design templates, learn creative skills or even hire freelancers. With an industry-leading marketplace paired with an unlimited subscription service, Envato helps creatives like you get projects done faster.\nAbout Envato\nCareers\nPrivacy Policy\nSitemap\nCommunity\nBlog\nForums\nMeetups\nFacebook\nTwitter\nYoutube\nPages\nABOUT US\nAMBER\nAnnual Return (Form MGT-7) as per Section 92(3)\

In [45]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [46]:
create_brochure(companyName, companyURL)

Found links: {'links': [{'type': 'about page', 'url': 'https://vjfurnishings.com/about-us-vj-furnishings/'}, {'type': 'careers page', 'url': 'https://vjfurnishings.com/careers/'}, {'type': 'corporate page', 'url': 'https://vjfurnishings.com/corporate/'}, {'type': 'infrastructure page', 'url': 'https://vjfurnishings.com/infrastructure/'}, {'type': 'network page', 'url': 'https://vjfurnishings.com/our-network/'}, {'type': 'news and events page', 'url': 'https://vjfurnishings.com/news-event/'}]}


# V&J Furnishings Brochure

---

## About Us

Welcome to **V&J Furnishings**, your go-to destination for high-quality upholstery, curtains, sheers, and artificial leather. We strive to provide our customers with a wide selection of innovative and stylish fabrics that cater to both indoor and outdoor needs. Our extensive product line is designed to enhance the aesthetic appeal of any space while delivering durable performance.

---

## Product Range

At V&J, we pride ourselves on our diverse catalog, which includes:

- **Upholstery Fabric**: A variety of textures and designs that redefine comfort and style.
- **Curtains & Sheers**: Elegant options that add a graceful touch to any window.
- **Leatherette**: Affordable alternatives that never compromise on style.
- **Outdoor/Indoor Fabrics**: Versatile materials designed for all environments.

Our featured brands such as **Amber**, **Boston**, and **Retro** bring you the latest trends in upholstery, ensuring your décor stays modern and fresh.

---

## Customers

V&J Furnishings serves a broad clientele, ranging from homeowners to interior designers and businesses. Our commitment to quality and rapid delivery has made us a trusted partner in the industry. Our clients have expressed their satisfaction with our collections, which have transformed their spaces in ways they never imagined.

> "Amazing Collection! We are very happy to select V&J Furnishings upholstery fabrics & curtains... the results come out awesome!" - **Rajiv K.**

---

## Company Culture

At V&J, we believe that a strong company culture drives success. We embrace creativity, inclusion, and collaboration within our teams. Our work environment encourages innovation and supports the professional growth of our employees, making it a great place for passionate individuals to thrive.

---

## Careers

Join the V&J family! We are always on the lookout for talented individuals who share our passion for design and commitment to quality. Explore our **Careers** page to discover exciting employment opportunities where you can make a difference in the world of furnishings.

---

## Contact Us

For more information about our products, partnerships or career opportunities, please visit our website or contact us directly. 

Connect with us:
- **Website**: [V&J Furnishings](#) 
- **Facebook**: [Facebook](#) 
- **Twitter**: [Twitter](#) 
- **YouTube**: [YouTube](#)

---

**V&J Furnishings** - Transforming Spaces with Style and Comfort!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [47]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [48]:
stream_brochure(companyName, companyURL)

Found links: {'links': [{'type': 'about page', 'url': 'https://vjfurnishings.com/about-us-vj-furnishings/'}, {'type': 'careers page', 'url': 'https://vjfurnishings.com/careers/'}, {'type': 'corporate page', 'url': 'https://vjfurnishings.com/corporate/'}, {'type': 'infrastructure page', 'url': 'https://vjfurnishings.com/infrastructure/'}, {'type': 'news and events page', 'url': 'https://vjfurnishings.com/news-event/'}, {'type': 'our network page', 'url': 'https://vjfurnishings.com/our-network/'}]}


# V&J Furnishings Company Brochure

---

## About V&J Furnishings

V&J Furnishings specializes in providing high-quality upholstery fabrics, curtains & sheers, and artificial leather products to decorate and enhance your living spaces. With a commitment to quality and design, we strive to transform interiors into elegant and inviting spaces, ensuring customer satisfaction at every step.

---

## Products We Offer

- **Upholstery Fabrics:** A diverse selection suited for every style and need.
- **Curtains & Sheers:** From classic to contemporary designs that add elegance to your rooms.
- **Artificial Leather:** Durable and stylish leatherette options perfect for various uses.
- **Outdoor/Indoor Solutions:** Weather-resistant textiles perfect for any environment.

---

## Our Brands

Our exclusive brand, **AMBER**, leads the upholstery revolution with innovative designs and sustainable materials. We also offer a range of products under various curated lines, including Urban Textures, Inspired Motifs, and Eco-friendly collections.

---

## Customer Testimonials

> “Amazing Collection! We are very happy to select V&J Furnishings upholstery fabrics & curtains from their wide collections. After completion of our home interior, results come out awesome which we never dreamt of. We highly recommend this brand for décor fabrics.” — **Rajiv K.**

> “Superb Product & Delivery Support! We have been doing business with V&J for the last couple of years, thanks to their product quality and fast delivery.” — **Satisfied Customer**

---

## Company Culture

At V&J Furnishings, we believe in fostering a collaborative and innovative work environment. Our team is dedicated to pushing boundaries in design while prioritizing sustainability and customer satisfaction. We value creativity and encourage our employees to bring their unique ideas to the table, enhancing our product development and operational efficiency.

---

## Careers

V&J Furnishings is always on the lookout for talented and passionate individuals to join our growing team. Whether you are a creative designer, a skilled craftsman, or a dynamic sales professional, we invite you to apply and become part of our journey to redefine interior spaces.

[Explore Career Opportunities](#)

---

## Connect with Us

For more information about our products, shop locations, and current offers:

- **Website:** [V&J Furnishings](#)
- **Contact Us:** Reach out through our website or visit our store locator.

---

Make your interior dreams come true with V&J Furnishings – your partner in creating beautiful and functional spaces! 

--- 

Join us today in the upholstery revolution!

In [49]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure(companyName, companyURL)

Found links: {'links': [{'type': 'about page', 'url': 'https://vjfurnishings.com/about-us-vj-furnishings/'}, {'type': 'careers page', 'url': 'https://vjfurnishings.com/careers/'}, {'type': 'infrastructure page', 'url': 'https://vjfurnishings.com/infrastructure/'}, {'type': 'brands page', 'url': 'https://vjfurnishings.com/brands/'}, {'type': 'corporate page', 'url': 'https://vjfurnishings.com/corporate/'}, {'type': 'contact page', 'url': 'https://vjfurnishings.com/contact-us/'}, {'type': 'our network page', 'url': 'https://vjfurnishings.com/our-network/'}, {'type': 'news and events page', 'url': 'https://vjfurnishings.com/news-event/'}]}


# V&J Furnishings Brochure

## About V&J Furnishings
V&J Furnishings is your premier destination for high-quality upholstery, curtains, sheers, and artificial leather. Catering to a diverse global market, we are dedicated to transforming living spaces with our innovative and stylish fabric solutions. Our expansive range includes everything from practical outdoor and indoor fabrics to sophisticated upholstery textiles, making us a versatile choice for both personal and commercial projects.

## Our Products
- **Upholstery Fabric**: A diverse collection perfect for enhancing the elegance of any space.
- **Curtains & Sheers**: Elegant window treatments that blend functionality with aesthetic appeal.
- **Leatherette**: Durable and stylish, backed by the latest technology in fabric innovation.
- **Outdoor/Indoor Fabrics**: Robust materials designed for both home interiors and outdoor spaces.

### Featured Brands
- **Amber**: Leading the upholstery revolution with cutting-edge designs and sustainable practices.
- **Urban Textures**: Sophisticated patterns inspired by city life.
- **Essence Matt Jacquards**: High-quality jacquard fabrics that bring depth and richness.

## Company Culture
At V&J Furnishings, we pride ourselves on cultivating a dynamic and inclusive company culture. Our team is composed of skilled professionals who are passionate about design and customer service. We encourage creativity, innovation, and collaboration among all team members, fostering an environment where ideas can flourish and employees can grow.

### Commitment to Sustainability
We recognize the importance of sustainability in our industry, and as a result, we continuously strive to minimize our impact on the environment. This commitment is evident in our product sourcing and manufacturing processes, ensuring that our materials are not only stylish but also eco-friendly.

## Our Customers
Our clientele includes interior designers, architects, and individual homeowners who appreciate quality and design. With a focus on delivering exceptional products and customer service, we build long-lasting relationships with our customers, helping them create beautiful spaces that reflect their unique tastes and lifestyles.

## Careers
Join our growing team at V&J Furnishings! We are always on the lookout for talented individuals who share our passion for design and excellence. We offer competitive salaries, benefits, and a supportive work environment. Interested candidates can find more information about open positions on our [Careers page](#).

### Testimonials
> "Amazing Collection! We are very happy to select V&J Furnishings upholstery fabrics & curtains from their wide collections. The results exceeded our expectations!" — Rajiv K.

> "Superb Product & Delivery Support! We have been working with V&J for years because of their high product quality and fast delivery." — Satisfied Customer

## Contact Us
For inquiries about our products or to learn more about our services, please visit our [Contact Us page](#).

### Connect with Us
Follow us on social media to stay updated on new arrivals, promotions, and design inspiration:
- Facebook: [V&J Facebook](#)
- Twitter: [V&J Twitter](#)
- YouTube: [V&J YouTube](#)

---
Thank you for considering V&J Furnishings! Let us help you bring your interior design visions to life with our exceptional products and unwavering commitment to quality.



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>